In [1]:
import sqlite3
import sys
import requests_cache
import pandas as pd
import requests
import random
import string
from bs4 import NavigableString
from bs4 import BeautifulSoup as bs
import re

In [2]:
sys.path.insert(0, "/Users/samdillard/python/github.com/samhld/nba_stats/app/")
requests_cache.install_cache()

In [55]:
from scraper import getters
from PandasBasketball.stats import player_gamelog
from PandasBasketball.stats import get_data_master
from preprocessor.builders import create_player_stats_df, create_all_player_stats_df
from dbs.models import Player
from scraper.getters import get_all_player_tables, get_full_player_urls, _get_player_table, PLAYER_BASE_URL
from preprocessor.mungers import to_numeric

In [4]:
url = "https://www.basketball-reference.com/players/j/jamesle01.html"
res = requests.get(url)
# bron_log = player_gamelog(res)

In [5]:
soup = bs(res.text, "lxml")
all_url_lists = soup.find("div", id="inner_nav").find_all("ul", class_="")
urls = []
for url_list in all_url_lists:
    for url in url_list.find_all("li"):
        urls.append(url.find("a")["href"])
urls

['/players/j/jamesle01/lebron-high-school-amateur-exhibition.html',
 '/players/j/jamesle01/gamelog/2004',
 '/players/j/jamesle01/gamelog/2005',
 '/players/j/jamesle01/gamelog/2006',
 '/players/j/jamesle01/gamelog/2007',
 '/players/j/jamesle01/gamelog/2008',
 '/players/j/jamesle01/gamelog/2009',
 '/players/j/jamesle01/gamelog/2010',
 '/players/j/jamesle01/gamelog/2011',
 '/players/j/jamesle01/gamelog/2012',
 '/players/j/jamesle01/gamelog/2013',
 '/players/j/jamesle01/gamelog/2014',
 '/players/j/jamesle01/gamelog/2015',
 '/players/j/jamesle01/gamelog/2016',
 '/players/j/jamesle01/gamelog/2017',
 '/players/j/jamesle01/gamelog/2018',
 '/players/j/jamesle01/gamelog/2019',
 '/players/j/jamesle01/gamelog/2020',
 '/players/j/jamesle01/gamelog/2021',
 '/players/j/jamesle01/gamelog-playoffs/',
 '/players/j/jamesle01/splits/2004',
 '/players/j/jamesle01/splits/2005',
 '/players/j/jamesle01/splits/2006',
 '/players/j/jamesle01/splits/2007',
 '/players/j/jamesle01/splits/2008',
 '/players/j/jamesle

In [6]:
from dataclasses import dataclass

In [7]:
# bron = Player(urls=urls)
# pro_gamelogs = sorted(list(set(filter(lambda x: "gamelog/" in x, urls))))
# pro_gamelogs
# pro_gamelogs_playoffs = sorted(list(set(filter(lambda x: "gamelog-" in x, urls))))
# pro_gamelogs_playoffs
pattern = re.compile(r'splits\/\d+')

In [8]:
urls = sorted(list(set(urls)))
# matches = []
# for url in urls:
#     match = pattern.search(url)
#     if match:
#         print(match.string)
splits_overall_pattern = re.compile(r'splits/$')
for url in urls:
    match = splits_overall_pattern.search(url)
    if match:
        print(match.string)


/players/j/jamesle01/splits/


In [9]:
# soup = bs(res.text, "lxml")
# all_url_lists = soup.find("div", id="inner_nav").find_all("ul", class_="")
# urls = []
# for url_list in all_url_lists:
#     for url in url_list.find_all("li"):
#         urls.append(url.find("a")["href"])
url = "https://www.basketball-reference.com/players/j/jamesle01.html"

def get_player_detail_urls(player_url):
    """
    Returns list of all player-specifc URLs found near bottom of player page.
    Includes URLs for gamelogs, splits, shooting, lineups, and on-off stats.
    """
    res = requests.get(player_url)
    soup = bs(res.text, "lxml")
    all_url_lists = soup.find("div", id="inner_nav").find_all("ul", class_="")
    urls = []
    for url_list in all_url_lists[:5]:
        for url in url_list.find_all("li"):
            urls.append(url.find("a")["href"])
    return urls

In [10]:
get_player_detail_urls(url)[:10]

['/players/j/jamesle01/lebron-high-school-amateur-exhibition.html',
 '/players/j/jamesle01/gamelog/2004',
 '/players/j/jamesle01/gamelog/2005',
 '/players/j/jamesle01/gamelog/2006',
 '/players/j/jamesle01/gamelog/2007',
 '/players/j/jamesle01/gamelog/2008',
 '/players/j/jamesle01/gamelog/2009',
 '/players/j/jamesle01/gamelog/2010',
 '/players/j/jamesle01/gamelog/2011',
 '/players/j/jamesle01/gamelog/2012']

In [11]:
# def split_detail_urls(urls):
#     """Returns tuple of lists of URLs by category.
#         Takes a list of URLs; the return of `get_player_detail_urls()`"""
    
#     pro_gamelogs = sorted(list(set(filter(lambda x: "gamelog/" in x, urls))))
#     pro_gamelogs_playoffs = sorted(list(set(filter(lambda x: "gamelog-" in x, urls))))
    
#     splits_by_year = []
#     splits_by_year_pattern = re.compile(r'splits\/\d+')
#     for url in urls:
#         match = pattern.search(url)
#         if match:
#             splits_by_year.append(match.string)
            
#     overall_splits_pattern = re.compile(r'splits/')
#     overall_splits = []
#     for url in urls:
#         match = splits_overall_pattern.search(url)
#         if match:
#             overall_splits.append(match.string)
    
#     shooting = sorted(list(set(filter(lambda x: "shooting/" in x, urls))))
    
#     lineups = sorted(list(set(filter(lambda x: "lineups/" in x, urls))))
    
#     on_off = sorted(list(set(filter(lambda x: "on-off/" in x, urls))))
    
#     return (pro_gamelogs, pro_gamelogs_playoffs, splits_by_year, overall_splits, shooting, lineups, on_off)

In [12]:
from scraper.getters import get_player_detail_paths, split_detail_paths

In [21]:
url = "https://www.basketball-reference.com/players/j/jamesle01.html"
pro_gamelogs, pro_gamelogs_playoffs, splits_by_year, overall_splits, shooting, lineups, on_off = split_detail_paths(get_player_detail_paths(url))
pro_gamelogs_playoffs

['/players/j/jamesle01/gamelog-playoffs/']

In [227]:
gamelog_url = "https://www.basketball-reference.com/players/j/jamesle01/gamelog/2004"
gamelog_res = requests.get(gamelog_url)
gamelog_df = player_gamelog(gamelog_res)

In [166]:
gamelog_df = gamelog_df.drop(gamelog_df.columns[4], axis=1)
to_numeric(gamelog_df)
gamelog_df.dtypes
index = pd.DatetimeIndex(gamelog_df["Date"])
gamelog_df.set_index(index, inplace=True)

In [218]:
gamelog_df.groupby(gamelog_df.index.month).mean()

,G,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
Date,,,,,,,,,,,,,,,,,,,,,
1,38.0,8.181818,22.000000,0.368455,0.545455,3.090909,0.142455,4.181818,5.818182,0.692545,...,4.272727,5.272727,5.818182,1.454545,0.545455,3.545455,2.181818,21.090909,11.781818,-3.000000
2,50.0,8.000000,19.384615,0.401538,0.692308,1.923077,0.355556,4.461538,5.538462,0.813538,...,4.538462,5.615385,4.692308,1.461538,1.000000,2.692308,1.692308,21.153846,14.546154,-2.384615
3,64.0,8.333333,19.000000,0.423933,0.733333,2.400000,0.290500,4.800000,6.533333,0.712786,...,3.066667,4.333333,5.933333,1.866667,1.000000,3.000000,1.866667,22.200000,16.320000,0.733333
4,75.5,7.125000,17.125000,0.409375,0.625000,1.875000,0.370000,6.000000,8.250000,0.767375,...,4.375000,5.250000,7.375000,2.250000,0.125000,3.625000,2.125000,20.875000,15.787500,-2.750000
10,1.5,10.000000,18.500000,0.535500,0.500000,3.500000,0.100000,2.500000,5.000000,0.452000,...,7.000000,9.000000,8.500000,2.500000,0.000000,4.500000,2.000000,23.000000,19.700000,-6.000000
11,10.0,6.400000,16.000000,0.382067,0.933333,2.600000,0.260714,3.066667,4.200000,0.732714,...,5.266667,6.733333,6.133333,1.000000,0.800000,3.666667,1.933333,16.800000,11.726667,-2.800000
12,25.0,8.666667,19.933333,0.424933,1.133333,4.066667,0.230786,4.800000,5.800000,0.819462,...,3.533333,5.000000,5.533333,1.933333,0.733333,4.066667,1.666667,23.266667,16.053333,-1.000000


In [173]:
gamelog_df.head()

,G,Date,Age,Tm,FG,FGA,FG%,3P,3PA,3P%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
Date,,,,,,,,,,,,,,,,,,,,,
2003-10-29,1.0,2003-10-29,18-303,CLE,12.0,20.0,0.600,0.0,2.0,0.0,...,4.0,6.0,9.0,4.0,0.0,2.0,3.0,25.0,24.7,-9.0
2003-10-30,2.0,2003-10-30,18-304,CLE,8.0,17.0,0.471,1.0,5.0,0.2,...,10.0,12.0,8.0,1.0,0.0,7.0,1.0,21.0,14.7,-3.0
2003-11-01,3.0,2003-11-01,18-306,CLE,3.0,12.0,0.250,0.0,1.0,0.0,...,4.0,4.0,6.0,2.0,0.0,2.0,3.0,8.0,5.0,-21.0
2003-11-05,4.0,2003-11-05,18-310,CLE,3.0,11.0,0.273,0.0,2.0,0.0,...,9.0,11.0,7.0,2.0,3.0,2.0,1.0,7.0,11.2,-3.0
2003-11-07,5.0,2003-11-07,18-312,CLE,8.0,18.0,0.444,1.0,2.0,0.5,...,5.0,5.0,3.0,0.0,0.0,7.0,2.0,23.0,9.0,-7.0


In [225]:
def create_player_detail_df(url):
#     if "gamelog/" in url:
    res = requests.get(url)
    df = player_gamelog(res)
    df = df.drop(df.columns[4], axis=1)
    to_numeric(df)
    index = pd.DatetimeIndex(df["Date"])
    df.set_index(index, inplace=True)

    return df

In [235]:
test_df = create_player_detail_df(gamelog_url)
test_df, test_df.index

(               G        Date     Age   Tm  Opp   GS     MP    FG   FGA    FG%  \
 Date                                                                            
 2003-10-29   1.0  2003-10-29  18-303  CLE  SAC  1.0  42:50  12.0  20.0  0.600   
 2003-10-30   2.0  2003-10-30  18-304  CLE  PHO  1.0  40:21   8.0  17.0  0.471   
 2003-11-01   3.0  2003-11-01  18-306  CLE  POR  1.0  39:10   3.0  12.0  0.250   
 2003-11-05   4.0  2003-11-05  18-310  CLE  DEN  1.0  41:06   3.0  11.0  0.273   
 2003-11-07   5.0  2003-11-07  18-312  CLE  IND  1.0  43:44   8.0  18.0  0.444   
 ...          ...         ...     ...  ...  ...  ...    ...   ...   ...    ...   
 2004-04-07  75.0  2004-04-07  19-099  CLE  MEM  1.0  36:35   4.0  11.0  0.364   
 2004-04-09  76.0  2004-04-09  19-101  CLE  MIA  1.0  42:13   6.0  14.0  0.429   
 2004-04-10  77.0  2004-04-10  19-102  CLE  MIA  1.0  31:38   4.0  11.0  0.364   
 2004-04-12  78.0  2004-04-12  19-104  CLE  MIL  1.0  45:18   9.0  20.0  0.450   
 2004-04-14  79.

In [80]:
gamelog_df.iloc[:,4]

Rk
1     @
2     @
3     @
4      
5     @
     ..
78    @
79    @
80     
81     
82    @
Name:  , Length: 82, dtype: object

In [14]:
def create_gamelog_df(url):
    res = requests.get(url)
    df = player_gamelog(res)
    df
    

In [15]:
bron=Player()

In [16]:
bron.pro_non_playoffs()

In [17]:
get_full_player_urls()

['http://www.basketball-reference.com/players/a/abdelal01.html',
 'http://www.basketball-reference.com/players/a/abdulza01.html',
 'http://www.basketball-reference.com/players/a/abdulka01.html',
 'http://www.basketball-reference.com/players/a/abdulma02.html',
 'http://www.basketball-reference.com/players/a/abdulta01.html',
 'http://www.basketball-reference.com/players/a/abdursh01.html',
 'http://www.basketball-reference.com/players/a/abernto01.html',
 'http://www.basketball-reference.com/players/a/ablefo01.html',
 'http://www.basketball-reference.com/players/a/abramjo01.html',
 'http://www.basketball-reference.com/players/a/abrinal01.html',
 'http://www.basketball-reference.com/players/a/achiupr01.html',
 'http://www.basketball-reference.com/players/a/ackeral01.html',
 'http://www.basketball-reference.com/players/a/ackerdo01.html',
 'http://www.basketball-reference.com/players/a/acresma01.html',
 'http://www.basketball-reference.com/players/a/actonbu01.html',
 'http://www.basketball-re

In [18]:
url = "https://www.basketball-reference.com/players/j/jamesle01.html"

res = requests.get(url)
soup = bs(res.text, "lxml")
all_suffix_lists = soup.find("div", id="inner_nav").find_all("ul", class_="")
suffixes = []
for suffix_list in all_suffix_lists[:5]: # only first 5 lists are relevant
    for suffix in suffix_list.find_all("li"):
        suffixes.append(suffix.find("a")["href"])
suffixes

['/players/j/jamesle01/lebron-high-school-amateur-exhibition.html',
 '/players/j/jamesle01/gamelog/2004',
 '/players/j/jamesle01/gamelog/2005',
 '/players/j/jamesle01/gamelog/2006',
 '/players/j/jamesle01/gamelog/2007',
 '/players/j/jamesle01/gamelog/2008',
 '/players/j/jamesle01/gamelog/2009',
 '/players/j/jamesle01/gamelog/2010',
 '/players/j/jamesle01/gamelog/2011',
 '/players/j/jamesle01/gamelog/2012',
 '/players/j/jamesle01/gamelog/2013',
 '/players/j/jamesle01/gamelog/2014',
 '/players/j/jamesle01/gamelog/2015',
 '/players/j/jamesle01/gamelog/2016',
 '/players/j/jamesle01/gamelog/2017',
 '/players/j/jamesle01/gamelog/2018',
 '/players/j/jamesle01/gamelog/2019',
 '/players/j/jamesle01/gamelog/2020',
 '/players/j/jamesle01/gamelog/2021',
 '/players/j/jamesle01/gamelog-playoffs/',
 '/players/j/jamesle01/splits/2004',
 '/players/j/jamesle01/splits/2005',
 '/players/j/jamesle01/splits/2006',
 '/players/j/jamesle01/splits/2007',
 '/players/j/jamesle01/splits/2008',
 '/players/j/jamesle

In [19]:
soup.find("div", id="innter_nav")